# 把 order 进行细节的拆分 拆到每一个order的细节 具备





1. order_id
1. order_date
1. order_datetime, 具体的日期，用于分析行为
2. order_user_id，留在后边用
3. order_provice
4. order_city
    这两个部分先按照省份来拆分
5. 


In [104]:
# 2023/02/07 只生成了部分 12月份的数据

In [2]:
from imports import *

In [3]:
# read order monthly data 

In [4]:
df = pd.read_csv(os.path.join(GE_DIR,'order_monthly.csv'))

In [5]:
df

,product_id,month,count_raw,year,product_name
0,1,2018-01-01,1057,2018,基础款
1,1,2018-02-01,997,2018,基础款
2,1,2018-03-01,1219,2018,基础款
3,1,2018-04-01,1029,2018,基础款
4,1,2018-05-01,1783,2018,基础款
...,...,...,...,...,...
139,4,2022-08-01,3418,2022,豪华款
140,4,2022-09-01,3460,2022,豪华款
141,4,2022-10-01,4003,2022,豪华款
142,4,2022-11-01,2870,2022,豪华款


In [6]:
week_ratio = [0.8,1,2,2.5,3.8,4,3.5]

In [7]:
# import relativedelta
from dateutil.relativedelta import relativedelta

In [21]:
# Generate random hour and minute according to the ratio of 24 hours

random_24_hour_rato = [1,1,1,1,1,1,2,3,4,8,7,6,3,6,5,4,3,3,2,4,7,8,12,7]
px.scatter(random_24_hour_rato)

In [24]:
# 按照平均，周末更多的方式分配订单

# iterate over df
for idx, row in df.iterrows():
    
    month_str = row['month']
    count_raw = int(row['count_raw'])
    
    first_day = datetime.strptime(month_str, '%Y-%m-%d')
    this_month = first_day.month
    last_day = first_day + relativedelta(months=1, days=-1)
    
    df_this_month = pd.DataFrame()
    dates_of_this_month = pd.date_range(first_day, last_day)
    df_this_month['date'] = dates_of_this_month

    week_days = dates_of_this_month.dayofweek + 1
    df_this_month['week_day'] = week_days
    
    # Generate order count for each week day according to week_ratio
    order_count = [count_raw * ratio / sum(week_ratio) for ratio in week_ratio]
    
    # Generate dict including day of week and order count
    _dict = dict(zip(week_days, order_count))
    
    # number of week days of each week_day
    df_this_month['num_week_day'] = df_this_month['week_day'].map(df_this_month['week_day'].value_counts())
    
    df_this_month['count_total'] = df_this_month['week_day'].map(_dict)
    
    df_this_month['count_orders'] = df_this_month['count_total'] / df_this_month['num_week_day']
    
    # Generate randomness for count_orders follow normal distribution
    df_this_month['count_orders_this_day'] = df_this_month['count_orders'].apply(lambda x: int(np.random.normal(x, x/5)))
    


In [25]:
df_this_month

,date,week_day,num_week_day,count_total,count_orders,count_orders_this_day
0,2022-12-01,4,5,169.818182,33.963636,46
1,2022-12-02,5,5,212.272727,42.454545,41
2,2022-12-03,6,5,424.545455,84.909091,81
3,2022-12-04,7,4,530.681818,132.670455,118
4,2022-12-05,1,4,806.636364,201.659091,176
5,2022-12-06,2,4,849.090909,212.272727,113
6,2022-12-07,3,4,742.954545,185.738636,197
7,2022-12-08,4,5,169.818182,33.963636,28
8,2022-12-09,5,5,212.272727,42.454545,35
9,2022-12-10,6,5,424.545455,84.909091,105


In [32]:
# Generate random hour and minute according to the ratio of 24 hours
def generate_hour_minute():
    hour = np.random.choice(range(24), p=np.array(random_24_hour_rato)/sum(random_24_hour_rato))
    minute = np.random.choice(range(60))
    return hour, minute

# Generate id with random hour and minute

In [47]:
# 

order_list = []

idx = 0
for idx, row in df_this_month.iterrows():

    for i in range(row['count_orders_this_day']):
        hour, minute = generate_hour_minute()
        
        order = {
            "id": idx,
            "date": row['date'].strftime('%Y-%m-%d'),
            "time": f"{hour:02d}:{minute:02d}",
            "datetime": row['date'].strftime('%Y-%m-%d') + f" {hour:02d}:{minute:02d}",
            
        }
        
        idx += 1
        
        order_list.append(order)

In [48]:
# change to df
df_order = pd.DataFrame(order_list)

In [49]:
df_order

,id,date,time,datetime
0,0,2022-12-01,22:45,2022-12-01 22:45
1,1,2022-12-01,13:47,2022-12-01 13:47
2,2,2022-12-01,10:39,2022-12-01 10:39
3,3,2022-12-01,10:40,2022-12-01 10:40
4,4,2022-12-01,15:43,2022-12-01 15:43
...,...,...,...,...
3539,106,2022-12-31,09:44,2022-12-31 09:44
3540,107,2022-12-31,13:49,2022-12-31 13:49
3541,108,2022-12-31,14:35,2022-12-31 14:35
3542,109,2022-12-31,08:04,2022-12-31 08:04


# 获取省份





In [51]:
# Generate list of all provices in China
provices = ['北京市','天津市','河北省','山西省','内蒙古自治区','辽宁省','吉林省','黑龙江省','上海市','江苏省','浙江省','安徽省','福建省','江西省','山东省','河南省','湖北省','湖南省','广东省','广西壮族自治区','海南省','重庆市','四川省','贵州省','云南省','西藏自治区','陕西省','甘肃省','青海省','宁夏回族自治区','新疆维吾尔自治区','台湾省','香港特别行政区','澳门特别行政区']

In [58]:
# generate all GDP data of all provices

# read order_propotion data

df_order_propotion = pd.read_excel(os.path.join(BASE_DIR,'order_propotion.xlsx'))

In [59]:
df_order_propotion

,provice,propotion
0,北京,0.10
1,上海,0.20
2,广州,0.11
3,深圳,0.10
4,江苏,0.30
5,浙江,0.25
6,山东,0.10


In [67]:
# generate random provice base on the propotion
def generate_provice(x):
    # make sum of propotion equal to 1
    df_order_propotion['propotion'] = df_order_propotion['propotion'] / sum(df_order_propotion['propotion'])
    
    provice = np.random.choice(df_order_propotion['provice'], p=df_order_propotion['propotion'].values)
    return provice

In [68]:
df_order['provice'] = df_order['id'].apply(generate_provice)

In [ ]:
# generate cily from provice


In [69]:
df_order

,id,date,time,datetime,provice
0,0,2022-12-01,22:45,2022-12-01 22:45,北京
1,1,2022-12-01,13:47,2022-12-01 13:47,江苏
2,2,2022-12-01,10:39,2022-12-01 10:39,江苏
3,3,2022-12-01,10:40,2022-12-01 10:40,江苏
4,4,2022-12-01,15:43,2022-12-01 15:43,上海
...,...,...,...,...,...
3539,106,2022-12-31,09:44,2022-12-31 09:44,上海
3540,107,2022-12-31,13:49,2022-12-31 13:49,上海
3541,108,2022-12-31,14:35,2022-12-31 14:35,江苏
3542,109,2022-12-31,08:04,2022-12-31 08:04,浙江


In [86]:
# read city data

df_city = pd.read_excel('/Users/zhitaogao/Library/CloudStorage/OneDrive-Personal/bbd-projects/mfg_operation_model/src/省市县区.xlsx')

In [91]:
# remove "省" and "市" from provice
df_city['provice'] = df_city['provice'].apply(lambda x: x.replace('省','').replace('市',''))

In [99]:
df_city['provice'].unique()

array(['北京', '天津', '河北', '山西', '内蒙古', '辽宁', '吉林', '黑龙江', '上海', '江苏', '浙江',
       '安徽', '福建', '江西', '山东', '河南', '湖北', '湖南', '广东', '广西', '海南', '重庆',
       '四川', '贵州', '云南', '西藏', '陕西', '甘肃', '青海', '宁夏', '新疆'], dtype=object)

In [92]:
df_city

,provice,city,县区
0,北京,北京市,东城区
1,北京,北京市,西城区
2,北京,北京市,崇文区
3,北京,北京市,宣武区
4,北京,北京市,朝阳区
...,...,...,...
2861,新疆,阿勒泰地区,吉木乃县
2862,新疆,省直辖行政单位,石河子市
2863,新疆,省直辖行政单位,阿拉尔市
2864,新疆,省直辖行政单位,图木舒克市


In [100]:
# Generate random city of each provice
def generate_city(x,df_city = df_city):
    try:
        df_this_provice = df_city[df_city['provice'] == x]
        city = np.random.choice(df_this_provice['city'])
    except:
        city = x
    return city

In [101]:
df_order['city'] = df_order['provice'].apply(generate_city)

In [102]:
df_order

,id,date,time,datetime,provice,city
0,0,2022-12-01,22:45,2022-12-01 22:45,北京,北京市
1,1,2022-12-01,13:47,2022-12-01 13:47,江苏,苏州市
2,2,2022-12-01,10:39,2022-12-01 10:39,江苏,扬州市
3,3,2022-12-01,10:40,2022-12-01 10:40,江苏,淮安市(原淮阴市）
4,4,2022-12-01,15:43,2022-12-01 15:43,上海,上海市
...,...,...,...,...,...,...
3539,106,2022-12-31,09:44,2022-12-31 09:44,上海,上海市
3540,107,2022-12-31,13:49,2022-12-31 13:49,上海,上海市
3541,108,2022-12-31,14:35,2022-12-31 14:35,江苏,泰州市
3542,109,2022-12-31,08:04,2022-12-31 08:04,浙江,温州市


In [103]:
# save as csv
df_order.to_csv(os.path.join(GE_DIR,'order_detail.csv'), index=False)